<a href="https://colab.research.google.com/github/luguzman/NLP/blob/main/BERT_Tokenizador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fase 1: Importar las dependencias

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub    # librería que nos ayuda a importar modelos con pesos ya almacenados

from tensorflow.keras import layers
import bert

# Fase 2: Pre procesado de datos

## Carga de los ficheros

Importamos los ficheros desde nuestro Google Drive personal.

In [ ]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/My Drive/Curso de NLP/BERT/sentiment_data/training.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)


In [ ]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [ ]:
data.head(6)

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
5,0,@Kwesidei not the whole crew


## Preprocessing

### Cleaning

In [ ]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Eliminar el @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Eliminar los links de la URL
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Conservamos solamente las letras
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Eliminamos espacios en blanco adicionales
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [ ]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [ ]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

### Tokenization

Necesitaremos crear una capa BERT para tener acceso a los meta datos para el tokenizador (como el tamaño del vocabulario).

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer # Instacia que nos permitirá convertir los textos a token
# Una vez teniendo este tokenizador necesitamos info adicional de este como cual es el tamaño del vocabulario,
# convertir todo a minúsculas, etc y por lo tanto aplicaremos la siguiente capa:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",   
                            trainable=False)
# en_uncased: english en minúscula
# L-12: Es la versión sencilla de BERT, para que sea rápido de entrenar vs el L-24
# trainable: Se utiliza para indicar si haremos find tuning, si hay que entrenar algo adicional de los propios pesos

# Extraemos el tamaño del vocabulario(file temporal que genera el hub) a partir del tokenizer. 
# Es más que el tamaño es el diccionario completo.
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
# Del mismo modo necesitamos la info de la conversión a minúsculas por parte de BERT.
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)
# Esto es normal que tarde pues con esta versión baja la arquitectura junto con los 110 M de pesos

In [ ]:
# Ejemplo de que podemos hacer con nuestro tokenizer
tokenizer.tokenize("My dog loves strawberries.")
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("My dog loves strawberries."))

In [ ]:
# Generamos una lista de tokens númericos, asociados a cada frase.
def encode_sentence(sent):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [ ]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

### Creación del data set

Crearemos padded batches (por lo que rellenamos las frases para cada lote de forma independiente), de esta forma añadimos el mínimo número de tokens de padding posible. Para eso, ordenamos las frases por longitud, aplicamos padded_batches y luego las mezclamos.

In [ ]:
# Primero creamos una variable la cual agrupará 3 cosas a la vez. De manero que lo haremos es empaquetar 
# de cada frase, la frase misma, su etiqueta, y su longitud
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
# Ahora necesitamos mezclarlas porque en el dataset original resulta que las frases han sido ordenadas 
# según la etiqueta que corresponde al sentimiento. Esto queire decir que la primera mitad (los primeros 
# 800 000 tweets) corresponden a frases con sentimiento negativo y la segunda mitad a positivos. Y lo 
# quiero mezclar ya que no quiero tener bloques donde tenga frases donde todo se negativo y/o positivo contantemente. 
random.shuffle(data_with_len)
# Utilizamos la función sort para ordenar de acuerdo a la longitud de la frase
data_with_len.sort(key=lambda x: x[2])
# Vamos a limpiar la información. Solo nos quedaremos la frase y su correspondiente sentimiento pero 
# unicamente si la frase tiene más de 7 palabras.
sorted_all = [(sent_lab[0], sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [ ]:
# En este punto nuestras aun no tiene todas la misma longitud por lo que utilizaremos un generator,
# para que se encargue de arreglarlas. Un generedor como funciona es simplemente le das un elemnto 
# y te duevuelve otro.  
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32)) # Tipo de dato de la salida (enteros en este caso)

In [ ]:
# iter: itera sobre los elementos de la lista. next: simplemente haría que nos devuelva la sig iteración.
# En este caso como le pusimo que se quedará con las frases con más de 7 palabras es de esperar que la primer
# que nos devolviera fuera de longitud 8
next(iter(all_dataset))

(<tf.Tensor: shape=(8,), dtype=int32, numpy=
 array([ 8038,  2100,   999,  4283,  2005,  1996, 21461,   999],
       dtype=int32)>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [ ]:
# Ahora haremos el proceso de Padding pero recordemos que será por bloques así reduciremos el entrenamiento
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
#next(iter(all_batched))

(<tf.Tensor: shape=(32, 8), dtype=int32, numpy=
 array([[ 8038,  2100,   999,  4283,  2005,  1996, 21461,   999],
        [ 4283,  2005,  1996,  2128,  2102, 28394,  2102,   999],
        [17012,  3566,  2056,  1045,  5807,  1005,  1056,  1012],
        [ 5760,  6199, 17299,  2003,  4569,  1998,  2025,  3733],
        [ 1057,  2074,  2561,  2100,  2081,  2026,  2154,   999],
        [ 1045,  3246,  1996,  6876,  1045,  7303,  2131,  2209],
        [ 2026,  2793,  1005,  1055,  2061,  4064, 22708,  1012],
        [ 2009,  1005,  1055,  2033,   999,   999,   999, 14089],
        [ 4931,  2045,  6898,   999,   999,  3407,  3407,  2420],
        [13442,  2909,  1012,  2507,  2028,  1037,  3046,   999],
        [14163,  3270,  3270,  3270,  3270,  2057,  2003,  3297],
        [ 4083,  2000,  2293,  2166,  1005,  1055,  8220,   999],
        [ 2003,  2061,  9479,  1012,  2009, 13403,  2061,  2172],
        [13183,  2696,  2031,  4569,  1999, 14068,  4665,   999],
        [12098, 13871,  5603

In [ ]:
# Generamos nustro conjunto de entrenamiento y testing por lotes.

# Declaramos el número total de lotes que va a haber.
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE)
# Declaramos el número total de lotes de test que va a haber.
NB_BATCHES_TEST = NB_BATCHES // 10  # Nos quedamos con el 10%
# SOlo recordemos que los lotes los tenemos ordenados por lo que si nos tomamos el 90% priemro para 
# entrenar y el 10% restante para testing me van a quedar el 10% más largo. Por lo que volveremos a 
# mezclar cada uno de los lotes.
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

# Fase 3: Construcción del modelo

In [ ]:
class DCNN(tf.keras.Model):
    # Declaramos el constructor
    def __init__(self,
                 vocab_size,    # Tamaño del vocabulario que vendrá dado por el tokenizador
                 emb_dim=128,   # Primera capa de embbedingo con una dimension de 128
                 nb_filters=50, # Número de filtros que le aplicaremos a RNC por cada vector de características
                 FFN_units=512, # Número de neuronas en la capa oculta. En este caso la penultima capa densa
                 nb_classes=2,  # Ya que es un proceso de clasificación ninario
                 dropout_rate=0.1,  # Apaga el 10% de las neuronas de manera aleatoria en cada epoch para evitar el overfitting
                 training=True,
                 name="dcnn"):
        # Para poder utlizar todas las varibales de tk.keras.Model lo primero y más importante tenemos que 
        # llamar a la super clase, ya que como hemos hecho que el método herede de tf.keras.Model. estamos 
        # obligados a llamar a :
        super(DCNN, self).__init__(name=name)   
        
        self.embedding = layers.Embedding(vocab_size,   # Recibe los números de los tokens y los convierte en vectores y estos vectores permiten que sean ajustado por la RN
                                          emb_dim)
        # Definimos nuestras redes de convolución. En este caso convoluiciones verticales (1D)
        # Definimos 3 familias de filtros diferentes. En este caso, filtros que analizarán 2,3 y 4 palabras juntas
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,      # analizamos las palabras cuando aparecen juntas de dos en dos, ie, filtrará palabras de 2 en 2.
                                    padding="valid",    # aqui no toma mucha importancia, lo que hará es añadir ceros en el caso de las primeras y ultimas convoluciones en las que ya no haya elementos. Pero como no hemos puesto el parametro de stride es = 1 por lo que no se podría salir
                                    activation="relu")  # trabaja muy bien en este tipo de análisis. La aplicamos para romper la linealidad y que busque más allá de valores lineales.
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        # Creamos una capa la cual se encarga de solo quedarse con el máximo de todo estos valores. El 
        # máximo de cada bi-tri-cuatrigrama
        self.pool = layers.GlobalMaxPool1D()
        
        # Pasamos estos maximos a la primera capa densa
        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x) # (batch_size, nb_filters, seq_len-1) Aplicará la transformación de bigramas a x
        x_1 = self.pool(x_1) # (batch_size, nb_filters) Ya que queremos quedarnos con el más grande, el más importante de esos valores de cada uno de los 50 de los mapas de características
        x_2 = self.trigram(x) # (batch_size, nb_filters, seq_len-2)
        x_2 = self.pool(x_2) # (batch_size, nb_filters)
        x_3 = self.fourgram(x) # (batch_size, nb_filters, seq_len-3)
        x_3 = self.pool(x_3) # (batch_size, nb_filters)
        
        # Concatenamos la lista formada por x1,x2 y x3 para formar nuestra macro entrada que pasará a nuestra red neuronal
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        # -1: representa el último de los ejes de combinacion. En este caso tenemos 2 ejes.
        # El primer eje es el de los bloques:
        # Recordemos que le damos batches de palabras. Por lo tanto el primer eje sería cada uno de los
        # objetos es un comentario.El segundo eje representa el propio valor que acabamos de sacar después
        # de cada uno de los max pooling.
        # Lo que se obtiene en la variable merge será un tensor cuyo tamaño será el número de filas, el
        # mismo que el del tamaño del bloque (batchsize), y el número de columnas será 3 veces * el numero filtros
        # 3 veces se debe a que habrá el número de filtros en bigramas, el número de filtros de trigramas, etc
        
        # Aplicamos la tranformación de la capa densa número 1 a toda esta info ya combinada
        merged = self.dense_1(merged)
        # Aplicamos la capa de Dropout unica y exclusivamente en fase de training 
        merged = self.dropout(merged, training)
        # Por último aplicamos la capa de salida 
        output = self.last_dense(merged)
        
        return output

# Fase 4: Entrenamiento

In [ ]:
# Los siguientes datos se definen a prueba y error buscando mejorar en los resultados

# Definimos el tamaño del vocabulario (lo limitamos)
VOCAB_SIZE = len(tokenizer.vocab) #

EMB_DIM = 200     # Todas y cada una de las palabras se mapeara a un espacio vectorial de dimensión 200 o lo que es lo mismo, cada palabra se identificará de forma única de con un punto de 200 coordenadas.
NB_FILTERS = 100  # Número de filtros de la red neuronal convolucional
FFN_UNITS = 256   # Número de unidades que la capa de Feed Forward tendrá en la capa oculta
NB_CLASSES = 2    # len(set(train_labels))

DROPOUT_RATE = 0.2  # Definimos la tasa de olvido

NB_EPOCHS = 5     # Número de veces que pasarémos por todo el conjunto de entrenamiento

In [ ]:
# Creamos la instancia del objeto DCNN
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [ ]:
# Compilamos el modleo de acuerdo al número de clases
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [ ]:
# Crearemos un sistema de checkpoint en google colab que en caso de que se reinicie la sesión podemos reanudar
# desde elúltimo checkpoint que se haya guardado o incluso más adelante , seguir entrenando con más textos 
# desde donde lo habíamos dejado en lugar de tener que crear la red neuronal desde el inicio.

# Definimos una ruta dentro de mi Google Drive donde se guardarán los checkpoints
checkpoint_path = "./drive/My Drive/Curso NLP/BERT/ckpt_bert_tok/"

# Guardamos el modelo que queremos guradar autoamticamente siempre que sea posible
ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

# Definimos el manager que será el encargado de guardar los checkpoints en la ruta establecida
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)
# max_to_keep: Es para definir que queremos que siempre se guarden los últimos 5 checkpoints.
# En este caso si quisieramso guardar todos bastaría con poner max_to_keep = 5 pues son 5 epochs

# Las siguientes líneas de código lo que hacen es preguntarle al Checkpoint Manager si hay o no hay 
# último checkpoint
if ckpt_manager.latest_checkpoint:  # esta linea devuelve un None si no hay checkpoint previo (If None:)
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

In [ ]:
# Función que entrega al método fit. Como se puede apreciar se le pueden entregar uno o más callbacks 
# y el objetivo es que se ejecuten algunas líneas de código entre cada epoch, al inicio de un epoch,
# al final, etc. Hay una serie de metodo que podemos sobreescribir. En este caso on_epoch_end, cuando 
# termine un epoch lo que quiero es que se ejecute el guardado que hemos dicho en el checkpoint manager.

# En otras palabras para que cada que fianlice un epoch haya un guardado de los pesos, de esos parámetros
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint guardado en {}.".format(checkpoint_path))
# logs=None: No queremos que se muestre ningún log en particular 

In [ ]:
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallback()])    # En este caso solo llamamos un callback, sin embargo, en esta lista podríamos llamar a 2,3, etc.

Epoch 1/5
  37196/Unknown - 1258s 34ms/step - loss: 0.4292 - accuracy: 0.8019Checkpoint guardado en ./drive/My Drive/Curso de NLP/BERT/ckpt_bert_tok/.
37196/37196 [==============================] - 1258s 34ms/step - loss: 0.4292 - accuracy: 0.8019
Epoch 2/5
37196/37196 [==============================] - 1223s 33ms/step - loss: 0.3814 - accuracy: 0.8301
Epoch 3/5
37196/37196 [==============================] - 1211s 33ms/step - loss: 0.3422 - accuracy: 0.8506
Epoch 4/5
37196/37196 [==============================] - 1224s 33ms/step - loss: 0.3017 - accuracy: 0.8710
Epoch 5/5
37196/37196 [==============================] - 1221s 33ms/step - loss: 0.2638 - accuracy: 0.8879


# Fase 5: Evaluación

In [ ]:
results = Dcnn.evaluate(test_dataset)
print(results)

4132/4132 [==============================] - 25s 6ms/step - loss: 0.4434 - accuracy: 0.8242
[0.4433819651603699, 0.8241847157478333]


In [ ]:
print(results)

[0.4433819651603699, 0.8241847157478333]




*   Training: 88.5%
*   Testing: 84.6%



In [ ]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)
    # Recordemos que cuando entrenamos el modelo nostros le pasabamos bloques de frase y si me quedo 
    # unicamente con con lo de arriba no sería un bloque por lo añadimos una dimesnion adiconal
    inputs = tf.expand_dims(tokens, 0)

    output = Dcnn(inputs, training=False)

    # El sentimiento lo obtendremos de redondear a la baja el output * 2. Esto ya que el número esta entre 0
    # y 1 y este al multiplicarlo * 2 estará entre 0 y 2. Al redondear a la baja toda cosa entre 0 y .999
    # será redondeado a 0 y todo valor entre 1 y 1.999 será 1 es imposible obtener 2 así que no hay problema
    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Salida del modelo: {}\nSentimiento predicho: Negativo.".format(
            output))
    elif sentiment == 1:
        print("Salida del modelo: {}\nSentimiento predicho: Positivo.".format(
            output))

In [ ]:
get_prediction("This movie was pretty interesting.")

Salida del modelo: [[0.99801946]]
Sentimiento predicho: Positivo.


In [ ]:
get_prediction("I'd rather not do that again.")

Salida del modelo: [[0.0191755]]
Sentimiento predicho: Negativo.
